In [39]:
import os
from google.cloud import bigquery
from dotenv import load_dotenv
import plotly.graph_objects as go

In [40]:

# Load environment variables
load_dotenv()

# Set up credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.getenv('SERVICE_ACCOUNT_FILE')

# Create a client
client = bigquery.Client()

In [41]:
import os
from google.cloud import bigquery
import plotly.graph_objects as go

# Get table details from environment variables
project_id = os.getenv('PROJECT_ID')
dataset_id = os.getenv('DATASET_NAME')
table_id = os.getenv('FEEDBACK_TABLE_NAME')

# Construct the query based on the actual schema
query = f"""
SELECT Label, COUNT(*) as count
FROM `{project_id}.{dataset_id}.{table_id}`
GROUP BY Label
ORDER BY Label
"""

In [42]:

# Create a BigQuery client
client = bigquery.Client()

# Run the query
query_job = client.query(query)  # Execute the query

# Fetch the results
results = query_job.result()  # Wait for the query to finish

# Prepare data for plotting
labels = []
values = []

# Process the results
for row in results:
    labels.append(row.Label)
    values.append(row.count)
    print(f"{row.Label}: {row.count}")

# create df from results
import pandas as pd
df = pd.DataFrame({'Label': labels, 'Count': values})

1: 2469
2: 2251
3: 5071
4: 18054
5: 79173


In [38]:
# analysis.py

import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from custom_theme import custom_theme

# Your existing code for creating the figure
fig = go.Figure(data=[go.Bar(
    x=labels,
    y=values,
    marker=dict(
        color=values,
        colorscale=px.colors.sequential.Viridis,
        colorbar=dict(title="Count"),
    )
)])

# Customize the layout
fig.update_layout(
    title='Sentiment Analysis Results',
    xaxis_title='Sentiment',
    yaxis_title='Count',
    bargap=0.2,  # gap between bars
    template=custom_theme,  # Apply our custom theme
    margin=dict(t=100, r=100)  # Increase top and right margins
)

# Additional customization for better readability
fig.update_layout(
    title_font_size=20,
    xaxis_title_font_size=16,
    yaxis_title_font_size=16,
    legend_title_font_size=16
)

# Add '@seszele' marker at top right corner of the figure
fig.add_annotation(
    text='@seszele',
    xref='paper',
    yref='paper',
    x=1.1,
    y=1.1,
    xanchor='right',
    yanchor='top',
    showarrow=False,
    font=dict(
        family='monospace',
        size=14,
        color='#ffffff'
    ),
    bgcolor='rgba(0,0,0,0.5)',
    bordercolor='#ffffff',
    borderwidth=1,
    borderpad=4,
    opacity=0.8,
    clicktoshow='onoff',
    name='seszele_annotation'
)

# Add invisible annotation with link
fig.add_annotation(
    text='<a href="https://github.com/seszele">Visit @seszele on GitHub</a>',
    xref='paper',
    yref='paper',
    x=1.1,
    y=1.05,
    xanchor='right',
    yanchor='top',
    showarrow=False,
    font=dict(
        family='monospace',
        size=14,
        color='#ffffff'
    ),
    bgcolor='rgba(0,0,0,0.5)',
    bordercolor='#ffffff',
    borderwidth=1,
    borderpad=4,
    opacity=0.8,
    visible=False,
    name='github_link'
)

# Add clicktoshow functionality
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(
                    label="@seszele",
                    method="update",
                    args=[{"annotations[1].visible": True}],
                    args2=[{"annotations[1].visible": False}],
                ),
            ],
            visible=False,
        )
    ]
)

# Show the plot
fig.show()

## Estimate cost
- analyse how many tokens would be needed to tokenize the text using gpt4 

In [60]:
import os
from google.cloud import bigquery
import tiktoken

def estimate_tokens_and_cost(model="gpt-3.5-turbo"):
    # Get table details from environment variables
    project_id = os.getenv('PROJECT_ID')
    dataset_id = os.getenv('DATASET_NAME')
    table_id = os.getenv('FEEDBACK_TABLE_NAME')

    # Construct the query
    query = f"""
    SELECT Review
    FROM `{project_id}.{dataset_id}.{table_id}`
    """

    # Create a BigQuery client
    client = bigquery.Client()

    # Run the query
    query_job = client.query(query)

    # Fetch the results
    results = query_job.result()

    # Initialize tokenizer
    enc = tiktoken.encoding_for_model(model)
    
    total_tokens = 0
    review_count = 0
    
    # Process the results
    for row in results:
        review = row.Review  # Changed from row.Text to row.Review
        tokens = len(enc.encode(review))
        total_tokens += tokens
        review_count += 1
    
    avg_tokens_per_review = total_tokens / review_count if review_count > 0 else 0
    
    # Pricing (as of September 2023)
    if model == "gpt-3.5-turbo":
        price_per_1k_tokens = 0.0015  # $0.0015 per 1K tokens for input
    elif model == "gpt-4":
        price_per_1k_tokens = 0.03  # $0.03 per 1K tokens for input
    else:
        raise ValueError("Unsupported model")
    
    estimated_cost = (total_tokens / 1000) * price_per_1k_tokens
    
    return {
        "total_reviews": review_count,
        "total_tokens": total_tokens,
        "avg_tokens_per_review": avg_tokens_per_review,
        "estimated_cost": estimated_cost
    }

# Usage
model = "gpt-4"

results = estimate_tokens_and_cost(model)

print(f"Total reviews: {results['total_reviews']}")
print(f"Total tokens: {results['total_tokens']}")
print(f"Average tokens per review: {results['avg_tokens_per_review']:.2f}")
print(f"Estimated cost: ${results['estimated_cost']:.2f}")

Total reviews: 107018
Total tokens: 3139660
Average tokens per review: 29.34
Estimated cost: $94.19
